# Implement Custom Layers


### Custom Layers
Custom layers are NN (Neural Network) layers that are not explicitly supported by a given framework. This tutorial demonstrates how to run inference on topologies featuring custom layers allowing you to plug in your own implementation for existing or completely new layers.

The list of known layers is different for any particular framework. To see the layers supported by the Intel® Distribution of OpenVINO™ toolkit, refer to the Documentation: https://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html#intermediate-representation-notation-catalog
<br><br>

*If your topology contains layers that are not in the list of known layers, the Model Optimizer considers them to be custom.*

The Model Optimizer searches for each layer of the input model in the list of known layers before building the model's internal representation, optimizing the model and producing the Intermediate Representation.

### Custom Layers implementation workflow in the Intel® Distribution of OpenVINO™ toolkit
When implementing the custom layer in the Intel® Distribution of OpenVINO™ toolkit for your pre-trained model, you will need to add extensions in both the Model Optimizer and the Inference Engine. The following figure shows the work flow for the custom layer implementation.
<br>

![image of CL workflow](./workflow.png)

### Example custom layer: Hyperbolic Cosine (cosh) function
We showcase custom layer implementation using a simple function; hyperbolic cosine (cosh). It's mathematically represented as:
![](gif.gif)

### Extension Generator
This tool generates extension source files with stubs for the core functions. To get the workable extension, you will add your implementation of these functions to the generated files.

### Steps to implement custom layers 

- Setup your environment for the Intel® Distribution of OpenVINO™ toolkit:<br>

In [ ]:
%%bash
source /opt/intel/openvino/bin/setupvars.sh

- Install prerequisites (code generator for running Python snippets):<br>

In [ ]:
! pip3 install cogapp

In [ ]:
! pip3 install opencv-python

### Create the TensorFlow* model (weights, graphs, checkpoints)
We create a simple model with a custom cosh layer. The weights are random and untrained, however sufficient for demonstrating Custom Layer conversion.

In [ ]:
! if [ -e "tf_model" ]; then echo "tf_model already exists"; else mkdir tf_model; fi

In [ ]:
! ./build_cosh_model.py tf_model

### Generate template files using the Extension Generator:

   We're using `$SV/custom-layer/extgen_output/` as the target extension path:<br><br>
   This will create templates that will be partially replaced by Python* and C++ code for executing the layer.

In [ ]:
! chmod +x create_model_ir.sh && ./create_model_ir.sh

When complete, the outputted text will appear similar to the following:
```
Stub file for TensorFlow Model Optimizer extractor is in /home/u30239/cl_tutorial/cl_cosh/user_mo_extensions/front/tf folder
Stub file for Model Optimizer operation is in cl_cosh/./user_mo_extensions/ops folder
Stub files for Inference Engine CPU extension are in cl_cosh/./user_ie_extensions/cpu folder
Stub files for Inference Engine GPU extension are in cl_cosh/./user_ie_extensions/gpu folder
```


#### Generate IR with custom layer using Model Optimizer
  We run the Model Optimizer for TensorFlow to convert and optimize the new model for the Intel® Distribution of OpenVINO™ toolkit. We explicitly set the batch to 1 because the model has an input dim of "-1". TensorFlow allows "-1" as a variable indicating "to be filled in later", but the Model Optimizer requires explicit information for the optimization process. The output is the full name of the final output layer.

### Inference Engine custom layer implementation for the Intel® CPU

- Copy CPU and GPU source code to the Model Optimizer extensions directory:<br>
   This will be used for building a back-end library for applications that implement cosh.

In [ ]:
! cd tf_model && mo_tf.py --input_meta_graph model.ckpt.meta --batch 1 --output "ModCosh/Activation_8/softmax_output" --extensions ../cl_cosh/user_mo_extensions --output_dir ../cl_ext_cosh

- Compile the C++ extension library:
Here we're building the back-end C++ library to be used by the Inference Engine for executing the cosh layer.

In [ ]:
! cp cosh_kernel_files/ext_cosh.cpp cl_cosh/user_ie_extensions/cpu/

In [ ]:
! cd cl_cosh/user_ie_extensions/cpu && cp ../../../CMakeLists.txt . && mkdir -p build && cd build && cmake ..

In [ ]:
! cd cl_cosh/user_ie_extensions/cpu/build && make -j $(nproc)

In [ ]:
! cp cl_cosh/user_ie_extensions/cpu/build/libcosh_cpu_extension.so cl_ext_cosh/

#### Note:
Make sure in your dev cloud account default OpenVINO samples has been builded. If not please navigate to /opt/intel/openvino/inference_engine/demos and run ./build_demos.sh

### Test your results:

 - <b>Using a C++ Sample:</b>

In [ ]:
! ~/inference_engine_samples_build/intel64/Release/classification_sample_async  -i ./dog.bmp -m cl_ext_cosh/model.ckpt.xml -d CPU -l cl_ext_cosh/libcosh_cpu_extension.so

- <b>Using a Python Sample:</b>

Try running the Python Sample without including the cosh extension library. You should see the error describing unsupported Cosh operation.

In [ ]:
! python3 /opt/intel/openvino/deployment_tools/inference_engine/samples/python_samples/classification_sample/classification_sample.py -i ./dog.bmp -m cl_ext_cosh/model.ckpt.xml -d CPU 

Now run the command with the cosh extension library:

In [ ]:
! python3 /opt/intel/openvino/deployment_tools/inference_engine/samples/python_samples/classification_sample/classification_sample.py -i ./dog.bmp -m cl_ext_cosh/model.ckpt.xml -d CPU -l cl_ext_cosh/libcosh_cpu_extension.so

### Inference Engine Custom Layer Implementation for the Intel® Integrated GPU

To run the custom layer on the GPU, the source code .cl and .xml files for the associated kernel must be added to the clDNN library and then specified on the command line of the sample. The .cl file is the OpenCL source code of the kernel that will be calculating the cosh custom layer. The .xml file describes the custom layer kernel to the Inference Engine GPU plugin so it may use the kernel with other layers in the clDNN library when running inference on the GPU.

- <b>Using a C++ Sample</b>

We will now run the command again, this time with the cosh extension kernel specified using the *"-c cosh_kernel.xml"* option to point to the cosh kernel in the command:

#### Writing a job file for submitting to a node

In [ ]:
%%writefile custom_layer_gpu.sh
cd $PBS_O_WORKDIR
~/inference_engine_samples_build/intel64/Release/classification_sample_async             -i dog.bmp \
                                                                                         -m cl_ext_cosh/model.ckpt.xml \
                                                                                         -d GPU \
                                                                                         -c cosh_kernel_files/cosh_kernel.xml 

In [ ]:
import os
job_id_core = !qsub custom_layer_gpu.sh -l nodes=1:tank-870:i5-6500te  -N custom_layer_gpu
print(job_id_core[0])
while True:
    var=job_id_core[0].split(".")
    file="custom_layer_gpu.o"+var[0]
    if os.path.isfile(file): 
        ! cat $file
        break
